## 8. Robustness and Performance

### 73 Know How to Use `heapq` for Priority Queues

In [1]:
import logging

In [2]:
class Book:
    def __init__(self, title, due_date):
        self.title = title
        self.due_date = due_date

In [3]:
def add_book(queue, book):
    queue.append(book)
    queue.sort(key=lambda x: x.due_date, reverse=True)

In [4]:
queue = []
add_book(queue, Book('Don Quixote', '2019-06-07'))
add_book(queue, Book('Frankenstein', '2019-06-05'))
add_book(queue, Book('Les Misérables', '2019-06-08'))
add_book(queue, Book('War and Peace', '2019-06-03'))

In [5]:
class NoOverdueBooks(Exception):
    pass

In [6]:
def next_overdue_book(queue, now):
    if queue:
        book = queue[-1]
        if book.due_date < now:
            queue.pop()
            return book

    raise NoOverdueBooks

In [7]:
now = '2019-06-10'

found = next_overdue_book(queue, now)
print(found.title)

found = next_overdue_book(queue, now)
print(found.title)

War and Peace
Frankenstein


In [8]:
def return_book(queue, book):
    queue.remove(book)

In [9]:
queue = []
book = Book('Treasure Island', '2019-06-04')

add_book(queue, book)
print('Before return:', [x.title for x in queue])

return_book(queue, book)
print('After return: ', [x.title for x in queue])

Before return: ['Treasure Island']
After return:  []


In [10]:
try:
    next_overdue_book(queue, now)
except NoOverdueBooks:
    pass          # Expected
else:
    assert False  # Doesn't happen

In [11]:
import random

In [12]:
import timeit

In [13]:
def print_results(count, tests):
    avg_iteration = sum(tests) / len(tests)
    print(f'Count {count:>5,} takes {avg_iteration:.6f}s')
    return count, avg_iteration

def print_delta(before, after):
    before_count, before_time = before
    after_count, after_time = after
    growth = 1 + (after_count - before_count) / before_count
    slowdown = 1 + (after_time - before_time) / before_time
    print(f'{growth:>4.1f}x data size, {slowdown:>4.1f}x time')

In [14]:
def list_overdue_benchmark(count):
    def prepare():
        to_add = list(range(count))
        random.shuffle(to_add)
        return [], to_add

    def run(queue, to_add):
        for i in to_add:
            queue.append(i)
            queue.sort(reverse=True)

        while queue:
            queue.pop()

    tests = timeit.repeat(
        setup='queue, to_add = prepare()',
        stmt=f'run(queue, to_add)',
        globals=locals(),
        repeat=100,
        number=1)

    return print_results(count, tests)

baseline = list_overdue_benchmark(500)
for count in (1_000, 1_500, 2_000):
    print()
    comparison = list_overdue_benchmark(count)
    print_delta(baseline, comparison)

Count   500 takes 0.001424s

Count 1,000 takes 0.004351s
 2.0x data size,  3.1x time

Count 1,500 takes 0.008618s
 3.0x data size,  6.1x time

Count 2,000 takes 0.015331s
 4.0x data size, 10.8x time


In [15]:
def list_return_benchmark(count):
    def prepare():
        queue = list(range(count))
        random.shuffle(queue)

        to_return = list(range(count))
        random.shuffle(to_return)

        return queue, to_return

    def run(queue, to_return):
        for i in to_return:
            queue.remove(i)

    tests = timeit.repeat(
        setup='queue, to_return = prepare()',
        stmt=f'run(queue, to_return)',
        globals=locals(),
        repeat=100,
        number=1)

    return print_results(count, tests)

baseline = list_return_benchmark(500)
for count in (1_000, 1_500, 2_000):
    print()
    comparison = list_return_benchmark(count)
    print_delta(baseline, comparison)

Count   500 takes 0.001156s

Count 1,000 takes 0.004571s
 2.0x data size,  4.0x time

Count 1,500 takes 0.010409s
 3.0x data size,  9.0x time

Count 2,000 takes 0.018811s
 4.0x data size, 16.3x time


In [16]:
from heapq import heappush

In [17]:
def add_book(queue, book):
    heappush(queue, book)

In [18]:
try:
    queue = []
    add_book(queue, Book('Little Women', '2019-06-05'))
    add_book(queue, Book('The Time Machine', '2019-05-30'))
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-18-c33d162d780d>", line 4, in <module>
    add_book(queue, Book('The Time Machine', '2019-05-30'))
  File "<ipython-input-17-7948396ce409>", line 2, in add_book
    heappush(queue, book)
TypeError: '<' not supported between instances of 'Book' and 'Book'


In [19]:
import functools

@functools.total_ordering
class Book:
    def __init__(self, title, due_date):
        self.title = title
        self.due_date = due_date

    def __lt__(self, other):
        return self.due_date < other.due_date

In [20]:
queue = []
add_book(queue, Book('Pride and Prejudice', '2019-06-01'))
add_book(queue, Book('The Time Machine', '2019-05-30'))
add_book(queue, Book('Crime and Punishment', '2019-06-06'))
add_book(queue, Book('Wuthering Heights', '2019-06-12'))
print([b.title for b in queue])

['The Time Machine', 'Pride and Prejudice', 'Crime and Punishment', 'Wuthering Heights']


In [21]:
queue = [
    Book('Pride and Prejudice', '2019-06-01'),
    Book('The Time Machine', '2019-05-30'),
    Book('Crime and Punishment', '2019-06-06'),
    Book('Wuthering Heights', '2019-06-12'),
]
queue.sort()
print([b.title for b in queue])

['The Time Machine', 'Pride and Prejudice', 'Crime and Punishment', 'Wuthering Heights']


In [22]:
from heapq import heapify

queue = [
    Book('Pride and Prejudice', '2019-06-01'),
    Book('The Time Machine', '2019-05-30'),
    Book('Crime and Punishment', '2019-06-06'),
    Book('Wuthering Heights', '2019-06-12'),
]
heapify(queue)
print([b.title for b in queue])

['The Time Machine', 'Pride and Prejudice', 'Crime and Punishment', 'Wuthering Heights']


In [23]:
from heapq import heappop

In [24]:
def next_overdue_book(queue, now):
    if queue:
        book = queue[0]           # Most overdue first
        if book.due_date < now:
            heappop(queue)        # Remove the overdue book
            return book

    raise NoOverdueBooks

In [25]:
now = '2019-06-02'

book = next_overdue_book(queue, now)
print(book.title)

book = next_overdue_book(queue, now)
print(book.title)

try:
    next_overdue_book(queue, now)
except NoOverdueBooks:
    pass          # Expected
else:
    assert False  # Doesn't happen

The Time Machine
Pride and Prejudice


In [26]:
def heap_overdue_benchmark(count):
    def prepare():
        to_add = list(range(count))
        random.shuffle(to_add)
        return [], to_add

    def run(queue, to_add):
        for i in to_add:
            heappush(queue, i)
        while queue:
            heappop(queue)

    tests = timeit.repeat(
        setup='queue, to_add = prepare()',
        stmt=f'run(queue, to_add)',
        globals=locals(),
        repeat=100,
        number=1)

    return print_results(count, tests)

baseline = heap_overdue_benchmark(500)
for count in (1_000, 1_500, 2_000):
    print()
    comparison = heap_overdue_benchmark(count)
    print_delta(baseline, comparison)

Count   500 takes 0.000209s

Count 1,000 takes 0.000395s
 2.0x data size,  1.9x time

Count 1,500 takes 0.000610s
 3.0x data size,  2.9x time

Count 2,000 takes 0.000803s
 4.0x data size,  3.9x time


In [27]:
@functools.total_ordering
class Book:
    def __init__(self, title, due_date):
        self.title = title
        self.due_date = due_date
        self.returned = False  # New field

    def __lt__(self, other):
        return self.due_date < other.due_date

In [28]:
def next_overdue_book(queue, now):
    while queue:
        book = queue[0]
        if book.returned:
            heappop(queue)
            continue

        if book.due_date < now:
            heappop(queue)
            return book

        break

    raise NoOverdueBooks

In [29]:
queue = []

book = Book('Pride and Prejudice', '2019-06-01')
add_book(queue, book)

book = Book('The Time Machine', '2019-05-30')
add_book(queue, book)
book.returned = True

book = Book('Crime and Punishment', '2019-06-06')
add_book(queue, book)
book.returned = True

book = Book('Wuthering Heights', '2019-06-12')
add_book(queue, book)

now = '2019-06-11'

book = next_overdue_book(queue, now)
assert book.title == 'Pride and Prejudice'

try:
    next_overdue_book(queue, now)
except NoOverdueBooks:
    pass          # Expected
else:
    assert False  # Doesn't happen

In [30]:
def return_book(queue, book):
    book.returned = True

assert not book.returned
return_book(queue, book)
assert book.returned

> - 우선순위 큐를 사용하면 선입선출이 아니라 원소의 중요도에 따라 원소를 처리할 수 있다.
> - 리스트 연산을 사용해 우선순위를 구현하면 큐 크기가 커짐에 따라 프로그램의 성능이 선형보다 더 빠르게 나빠진다.
> - `heapq` 내장 모듈은 효율적으로 규모 확장이 가능한 우선순위 큐를 구현하는 데 필요한 모든 기능을 제공한다.
> - `heapq`를 사용하려면 우선순위를 부여하려는 원소들이 자연스러운 순서를 가져야 한다. 이는 원소를 표현하는 클래스에 `__lt__`와 같은 특별 메서드가 있어야 한다는 뜻이다.

> - 스레드 안전한 다른 선택이 필요하다면 `queue.PriorityQueue` 클래스를 보라.